In [154]:
import pandas as pd

In [155]:
# load the datasets
utterance_df = pd.read_excel("../data/utterance_data.xlsx")
linguistic_df = pd.read_excel("../data/linguistic_outcomes.xlsx")
demographics_df = pd.read_excel("../data/demographic(1).xlsx")

In [156]:
# print(utterance_df.head())
# print(linguistic_df.head())
# print(demographics_df.head())

print(utterance_df.columns)
print(linguistic_df.columns)
print(demographics_df.columns)

Index(['Unnamed: 0', 'file', 'utterance', 'DX1', 'label', 'ncc', 'MOCATOTS',
       'MOCA_impairment', 'age', 'race', 'gender', 'educ'],
      dtype='object')
Index(['file', '# utterances (overall)', '# utterances (interviewer)',
       '# utterances (participant)', '# tokens (overall)',
       '# tokens (interviewer)', '# tokens (participant)',
       '# unique tokens (overall)', '# unique tokens (interviewer)',
       '# unique tokens (participant)',
       ...
       'MATTR (participant)', 'Maas (overall)', 'Maas (interviewer)',
       'Maas (participant)', 'MTLD (overall)', 'MTLD (interviewer)',
       'MTLD (participant)', 'HD-D (overall)', 'HD-D (interviewer)',
       'HD-D (participant)'],
      dtype='object', length=142)
Index(['REGTRYID', 'ADRC_COGNITIVE_DATA', 'ADRCVIS', 'Assessment.Date',
       'PTINIT', 'RID', 'VISCODE', 'FORMVER', 'SITEID', 'ENTRY',
       ...
       'LUMI_TAU_POS', 'tTau_AB42Positivity', 'pTau_AB42Ratio',
       'AB42_AB40Positivity', 'VAR00001', 'VAR00

In [157]:
# removing extra column
utterance_df = utterance_df.drop(columns=['Unnamed: 0'], errors='ignore')

print(utterance_df.columns)

Index(['file', 'utterance', 'DX1', 'label', 'ncc', 'MOCATOTS',
       'MOCA_impairment', 'age', 'race', 'gender', 'educ'],
      dtype='object')


In [158]:
# formatting the column names
utterance_df.columns = utterance_df.columns.str.strip().str.replace(' ', '')
linguistic_df.columns = linguistic_df.columns.str.strip().str.replace(' ', '').str.replace('#', '')

print(utterance_df.columns)
print(linguistic_df.columns)

Index(['file', 'utterance', 'DX1', 'label', 'ncc', 'MOCATOTS',
       'MOCA_impairment', 'age', 'race', 'gender', 'educ'],
      dtype='object')
Index(['file', 'utterances(overall)', 'utterances(interviewer)',
       'utterances(participant)', 'tokens(overall)', 'tokens(interviewer)',
       'tokens(participant)', 'uniquetokens(overall)',
       'uniquetokens(interviewer)', 'uniquetokens(participant)',
       ...
       'MATTR(participant)', 'Maas(overall)', 'Maas(interviewer)',
       'Maas(participant)', 'MTLD(overall)', 'MTLD(interviewer)',
       'MTLD(participant)', 'HD-D(overall)', 'HD-D(interviewer)',
       'HD-D(participant)'],
      dtype='object', length=142)


In [159]:
# subset of dataset with the necessary columns
utterance_df_subset = utterance_df[['file', 'utterance', 'DX1', 'MOCATOTS', 'MOCA_impairment', 'age', 'gender', 'educ']]
linguistic_df_subset = linguistic_df[['file', 'tokens(participant)', 'uniquetokens(participant)', 'AUX(participant)', 'CCONJ(participant)', 'NUM(participant)', 'PROPN(participant)', 'VERB(participant)', 'DATE(participant)', 'TTR(participant)', 'MATTR(participant)']]
demographic_df_subset = demographics_df[['REGTRYID', 'AB40_LUMI', 'AB42_LUMI', 'P_TAU_LUMI', 'T_TAU_LUMI', 'AB42_AB40Ratio', 'tTau_AB42Ratio', 'pTau_AB42Ratio']]

In [160]:
# check missing values for in utterance data
utterance_df_subset.isna().sum()

file               0
utterance          0
DX1                0
MOCATOTS           0
MOCA_impairment    0
age                8
gender             8
educ               8
dtype: int64

In [161]:
# to fill in missing values checking mean, median or mode is better in utterance data
numeric_cols = ['age', 'gender', 'educ']

for col in numeric_cols:
    mean_val = utterance_df_subset[col].mean()
    median_val = utterance_df_subset[col].median()
    print(f"{col}: mean = {mean_val:.2f}, median = {median_val:.2f}")

utterance_df_subset[numeric_cols].skew()

age: mean = 76.08, median = 75.00
gender: mean = 0.55, median = 1.00
educ: mean = 17.36, median = 18.00


age       0.749923
gender   -0.222190
educ     -0.250832
dtype: float64

Age is positively skewed, and the mean and median are almost the same, but since the median is slightly lower, the median will be used to fill missing values.
Gender is coded 0/1, so using the mode would be better for filling missing values.
Education (educ) is slightly negatively skewed, with a higher median; therefore, the median will be used to fill in missing values.

In [162]:
# filling the missing values in utterance data
cols = ['age', 'educ']
medians = utterance_df_subset[cols].median()
utterance_df_subset[cols] = utterance_df_subset[cols].fillna(medians)

mode_val = utterance_df_subset['gender'].mode()[0]
utterance_df_subset['gender'] = utterance_df_subset['gender'].fillna(mode_val)

utterance_df_subset.isna().sum()

/var/folders/4h/b8zy5xq90hxdqyzpsx60rng80000gn/T/ipykernel_44045/3333559885.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utterance_df_subset[cols] = utterance_df_subset[cols].fillna(medians)
/var/folders/4h/b8zy5xq90hxdqyzpsx60rng80000gn/T/ipykernel_44045/3333559885.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utterance_df_subset['gender'] = utterance_df_subset['gender'].fillna(mode_val)


file               0
utterance          0
DX1                0
MOCATOTS           0
MOCA_impairment    0
age                0
gender             0
educ               0
dtype: int64

In [163]:
# check missing values in linguistic data
linguistic_df_subset.isna().sum()

file                         0
tokens(participant)          4
uniquetokens(participant)    4
AUX(participant)             4
CCONJ(participant)           4
NUM(participant)             4
PROPN(participant)           4
VERB(participant)            4
DATE(participant)            4
TTR(participant)             4
MATTR(participant)           4
dtype: int64

In [164]:
# to fill in missing values checking mean, median or mode is better in linguistic data
numeric_cols = ['tokens(participant)', 'uniquetokens(participant)', 'AUX(participant)', 'CCONJ(participant)', 'NUM(participant)', 'PROPN(participant)', 'VERB(participant)', 'DATE(participant)', 'TTR(participant)', 'MATTR(participant)']

for col in numeric_cols:
    mean_val = linguistic_df_subset[col].mean()
    median_val = linguistic_df_subset[col].median()
    print(f"{col}: mean = {mean_val:.2f}, median = {median_val:.2f}")

linguistic_df_subset[numeric_cols].skew()

tokens(participant): mean = 659.30, median = 663.00
uniquetokens(participant): mean = 236.87, median = 248.50
AUX(participant): mean = 47.73, median = 48.50
CCONJ(participant): mean = 37.70, median = 35.00
NUM(participant): mean = 10.38, median = 10.00
PROPN(participant): mean = 21.76, median = 20.00
VERB(participant): mean = 75.90, median = 77.50
DATE(participant): mean = 5.19, median = 4.00
TTR(participant): mean = 0.42, median = 0.37
MATTR(participant): mean = 0.99, median = 0.99


tokens(participant)         -0.007291
uniquetokens(participant)   -0.415031
AUX(participant)             0.247956
CCONJ(participant)           0.366389
NUM(participant)             0.321662
PROPN(participant)           1.123676
VERB(participant)            0.169596
DATE(participant)            1.082661
TTR(participant)             2.252689
MATTR(participant)          -2.007696
dtype: float64

Tokens are symmetrical and almost 0, so the mean will be used to fill in missing values.\
Unique Tokens is slightly skewed, so the median is a better option.\
AUX is distributed symmetrically, so the mean will be used to fill in missing values.\
CCONJ is slightly skewed; the median will be used to fill in missing values.\
The NUM is skewed somewhat; therefore, the median will be used to fill in the missing values.\
PROPN is quite positively skewed, so the median would be a better option.\
VERB is almost symmetrical, and the mean will be used to fill in missing values.\
DATE is positively skewed, and the median will be used to fill in missing values.\
TTR is highly skewed, and the median would be a better option.\
MATTR is highly negatively skewed; therefore, the median will be used to fill in missing values.

In [165]:
# filling the missing values in linguistic data
df = linguistic_df_subset
mean_cols   = ['tokens(participant)', 'AUX(participant)', 'VERB(participant)']
median_cols = ['uniquetokens(participant)',
               'CCONJ(participant)', 'NUM(participant)',
               'PROPN(participant)', 'DATE(participant)',
               'TTR(participant)', 'MATTR(participant)']

# fill in mean columns
for col in mean_cols:
    mean_val = df[col].mean()
    df[col] = df[col].fillna(mean_val)

# fill in median columns
for col in median_cols:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)

linguistic_df_subset.isna().sum()

/var/folders/4h/b8zy5xq90hxdqyzpsx60rng80000gn/T/ipykernel_44045/3212703898.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(mean_val)
/var/folders/4h/b8zy5xq90hxdqyzpsx60rng80000gn/T/ipykernel_44045/3212703898.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(median_val)


file                         0
tokens(participant)          0
uniquetokens(participant)    0
AUX(participant)             0
CCONJ(participant)           0
NUM(participant)             0
PROPN(participant)           0
VERB(participant)            0
DATE(participant)            0
TTR(participant)             0
MATTR(participant)           0
dtype: int64

In [166]:
demographic_df_subset.isna().sum()

REGTRYID           0
AB40_LUMI         11
AB42_LUMI         11
P_TAU_LUMI        11
T_TAU_LUMI        11
AB42_AB40Ratio    11
tTau_AB42Ratio    11
pTau_AB42Ratio    11
dtype: int64

In [167]:
# to fill in missing values checking mean, median or mode is better in utterance data
numeric_cols = ['AB40_LUMI','AB42_LUMI','P_TAU_LUMI','T_TAU_LUMI','AB42_AB40Ratio','tTau_AB42Ratio','pTau_AB42Ratio']

for col in numeric_cols:
    mean_val = demographic_df_subset[col].mean()
    median_val = demographic_df_subset[col].median()
    print(f"{col}: mean = {mean_val:.2f}, median = {median_val:.2f}")

demographic_df_subset[numeric_cols].skew()

AB40_LUMI: mean = 11410.40, median = 11354.50
AB42_LUMI: mean = 875.04, median = 788.00
P_TAU_LUMI: mean = 46.33, median = 39.25
T_TAU_LUMI: mean = 348.26, median = 316.50
AB42_AB40Ratio: mean = 0.08, median = 0.09
tTau_AB42Ratio: mean = 0.60, median = 0.33
pTau_AB42Ratio: mean = 0.08, median = 0.04


AB40_LUMI         0.331881
AB42_LUMI         0.737590
P_TAU_LUMI        1.915302
T_TAU_LUMI        1.451436
AB42_AB40Ratio   -0.499330
tTau_AB42Ratio    7.975608
pTau_AB42Ratio    7.627163
dtype: float64

For AB40_LUMI, the mean and median are close, and the distribution is slightly skewed; therefore, the mean would be a better option.\
For AB42_LUMI, the mean is larger, and it is skewed; therefore, the median would be a better option.\
For P_TAU_LUMI, it is highly skewed, so the median would be a better option.
T_TAU_LUMI is highly skewed, so the median would be a better option.\
The AB42_AB40 ratio is negatively skewed, so the median would be a better option.\
Tau_AB42Ratio is very highly positively skewed; the median would be a better option.\
pTau_AB42Ratio is very highly positively skewed; the median would be a better option.

In [168]:
# filling the missing values in linguistic data
df = demographic_df_subset

mean_cols = ['AB40_LUMI']
median_cols = [
    'AB42_LUMI', 'P_TAU_LUMI', 'T_TAU_LUMI',
    'AB42_AB40Ratio', 'tTau_AB42Ratio', 'pTau_AB42Ratio'
]

for col in mean_cols:
    mean_val = df[col].mean()
    df[col] = df[col].fillna(mean_val)

for col in median_cols:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)
    
demographic_df_subset.isna().sum()

/var/folders/4h/b8zy5xq90hxdqyzpsx60rng80000gn/T/ipykernel_44045/308922001.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(mean_val)
/var/folders/4h/b8zy5xq90hxdqyzpsx60rng80000gn/T/ipykernel_44045/308922001.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(median_val)


REGTRYID          0
AB40_LUMI         0
AB42_LUMI         0
P_TAU_LUMI        0
T_TAU_LUMI        0
AB42_AB40Ratio    0
tTau_AB42Ratio    0
pTau_AB42Ratio    0
dtype: int64

In [169]:
# merging linguistics and utterance dataset for analyzing later
merged_df = pd.merge(utterance_df_subset, linguistic_df_subset, on='file', how='inner')

# a new column that matches the REGTRYID in demographics
merged_df['REGTRYID'] = merged_df['file'].str.split('_').str[0].astype(int)

# final merge with demographics
final_merged_df = pd.merge(
    merged_df,
    demographic_df_subset,
    on='REGTRYID',
    how='inner'
)

print(final_merged_df.columns)

Index(['file', 'utterance', 'DX1', 'MOCATOTS', 'MOCA_impairment', 'age',
       'gender', 'educ', 'tokens(participant)', 'uniquetokens(participant)',
       'AUX(participant)', 'CCONJ(participant)', 'NUM(participant)',
       'PROPN(participant)', 'VERB(participant)', 'DATE(participant)',
       'TTR(participant)', 'MATTR(participant)', 'REGTRYID', 'AB40_LUMI',
       'AB42_LUMI', 'P_TAU_LUMI', 'T_TAU_LUMI', 'AB42_AB40Ratio',
       'tTau_AB42Ratio', 'pTau_AB42Ratio'],
      dtype='object')


In [170]:
final_merged_df.describe()

,MOCATOTS,MOCA_impairment,age,gender,educ,tokens(participant),uniquetokens(participant),AUX(participant),CCONJ(participant),NUM(participant),...,TTR(participant),MATTR(participant),REGTRYID,AB40_LUMI,AB42_LUMI,P_TAU_LUMI,T_TAU_LUMI,AB42_AB40Ratio,tTau_AB42Ratio,pTau_AB42Ratio
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,...,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,19.255556,0.211111,75.766667,0.600000,17.400000,659.302326,237.388889,47.732558,37.577778,10.366667,...,0.414374,0.989129,3796.344444,11410.400000,865.366667,45.538889,344.733333,0.078479,0.566915,0.077772
std,2.497464,0.410383,5.205982,0.492642,2.015669,330.553335,94.221493,26.843041,21.338880,6.698063,...,0.137472,0.007830,843.527041,3240.589229,363.758083,24.268190,170.165059,0.023422,1.180730,0.165960
min,8.000000,0.000000,67.000000,0.000000,10.000000,7.000000,6.000000,0.000000,0.000000,0.000000,...,0.278298,0.955502,2764.000000,4599.000000,46.000000,14.300000,99.000000,0.005765,0.149340,0.022525
25%,18.000000,0.000000,72.250000,0.000000,16.000000,440.750000,185.750000,25.000000,24.000000,5.000000,...,0.338058,0.986332,3566.000000,9171.500000,608.000000,30.125000,226.750000,0.058727,0.250080,0.033274
50%,20.000000,0.000000,75.000000,1.000000,18.000000,660.151163,248.500000,47.732558,35.000000,10.000000,...,0.371693,0.989751,3684.000000,11410.400000,788.000000,39.250000,316.500000,0.086684,0.327180,0.038540
75%,21.000000,0.000000,79.000000,1.000000,18.000000,863.750000,300.250000,65.750000,52.750000,14.750000,...,0.430856,0.994091,3764.500000,13445.000000,1051.250000,50.100000,406.250000,0.093296,0.505736,0.062109
max,22.000000,1.000000,90.000000,1.000000,24.000000,1387.000000,441.000000,115.000000,106.000000,26.000000,...,0.941176,1.000000,8264.000000,22797.000000,2113.000000,163.400000,1012.000000,0.123842,11.217391,1.550000


In [171]:
# importing final merged df
final_merged_df.to_csv("data.csv", index=False)